# 04_6 OLS Model

Due to NDA agreements no data can be displayed.

Data Preparation, Data Cleaning, and Preparation for Modelling is the same for all algorithms. To directly go to modelling click [here](#modelling)

---

## Data preparation

### Import libraries and read data

In [ ]:
import pandas as pd 
import numpy as np

from sklearn.metrics import mean_squared_error 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
import mlflow
from modeling.config import EXPERIMENT_NAME
TRACKING_URI = open("../.mlflow_uri").read().strip()

In [ ]:
# read data
df = pd.read_csv('../data/Featureselection03.csv')
df.head()

### Create data frame with important features

So that everyone is on track with the feature selection, we created another csv file to rate the importance and only use important features for training our models and further analysis.

Only important features are used to train the model. In this case we use 17 features beside the target.

In [ ]:
# read list with feature importance
data_log = pd.read_csv('../data/Capstone_features_Features.csv')
data_log.head()

### Create data frame with important features

Only Features with a feature importance smaller than 3 were selected.

In [ ]:
# create list of important features (feature importance < 3)
list_imp_feat = list(data_log[data_log['ModelImportance'] < 3]['VarName'])
len(list_imp_feat)

In [ ]:
df_model = df[list_imp_feat].copy()

In [ ]:
df_model.info()

### Fill and drop NaN

Values for V.SLPOG.act.PRC and ME.SFCI.act.gPkWh contain missing values. The EDA showed that these are mainly caused during harbour times when the main engine was not running. Therefore it makes sense to fill the missing values with 0.

In [ ]:
df_model['V.SLPOG.act.PRC'].fillna(0,inplace=True)
df_model['ME.SFCI.act.gPkWh'].fillna(0,inplace=True)

The remaining rows with missing values are dropped.

In [ ]:
df_model.dropna(inplace=True)

In [ ]:
df_model.info()

### Check correlations

In [ ]:
plt.figure(figsize = (30,28))
sns.heatmap(df_model.corr(), annot = True, cmap = 'RdYlGn')

V.SOG.act.kn is still highly correlated with the target, but this feature is necessary to keep.

### Define Target

For this project the focus is on optimising the fuel consumption. Therefore the supply mass rate is used as target. Target values greater 8 t/h are defined as outlier.

In [ ]:
X = df_model.drop(['ME.FMS.act.tPh'], axis = 1)
y = df_model['ME.FMS.act.tPh']

### Train Test Split

Due to the high amount of data, a split into 10% test data and 90% train data is chosen. The random state is set to 42 to have comparable results for diffent models. To account for the imbalance in the distribution of passage types the stratify parameter is used for this feature. This results in approximately the same percentage of the different passage types in each subset.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = X['passage_type'], test_size = 0.1, random_state = 42)

### Create dummy values for passage type

Object types need to be transformed to dummy values. For this model this concerns the passage types.

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

### Add constant

For the OLS model an constant for the y axis section needs to be added.

In [ ]:
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

### Set MLFlow connection

MLFlow is used to track and compare different models and model settings.

In [ ]:
# setting the MLFlow connection and experiment
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.start_run(run_name='OLS_unscaled') # CHANGE!
run = mlflow.active_run()

---

## Modelling <a id='modelling'></a>

For all models in this project a MinMaxScaler is applied. For this model a random forrest is used. The hyperparameter are selected based on grid search and offer a reasonable balance between optimal results and overfitting. These settings are used in a pipeline.

In [ ]:
X_train.head()

Linear regression can be done with and without Feature scaling.

In [ ]:
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

Setting up the OLS model. In the OLS model the formular for a linear regression is being solved. Therefore we can afterwards have a look at the coefficients to understand feature importance.

In [ ]:
lin_reg = sm.OLS(y_train, X_train)

### Fit and predict

Training the OLS model.

In [ ]:
model = lin_reg.fit()

In [ ]:
model.summary()

Performing model predictions.

In [ ]:
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

---

## Analysis

### Errors and residuals

The root mean squared error (RMSE) is used to evaluate the model. 

In [ ]:
print('RMSE train: ', mean_squared_error(y_train, y_pred_train, squared= False))
rmse_train = mean_squared_error(y_train, y_pred_train, squared= False)
print('RMSE test: ', mean_squared_error(y_test, y_pred, squared= False))
rmse_test = mean_squared_error(y_test, y_pred, squared= False)

Plotting actual values against predicted shows that the points are close to the optimal diagonale. However, this plot and the yellowbrick residual plot show some dificulties the model has when predicting low target values.

In [ ]:
fig=plt.figure(figsize=(6, 6))
plt.axline([1, 1], [2, 2],color='lightgrey')
plt.scatter(y_train, y_pred_train, color ='#33424F')
plt.scatter(y_test, y_pred, color = '#FF6600')
#plt.xticks(np.arange(0,501,100));
#plt.yticks(np.arange(0,501,100));
plt.xlabel("ME.FMS.act.tPh actual");
plt.ylabel("ME.FMS.act.tPh predicted");
plt.xlim(-2, 8);
plt.ylim(-2, 8);

Here it can be seen, that especially while trying to predict lower values for the target the model makes some errors in both directions. For higher values the model underpredicts.

---

### Write to MLFlow

In [ ]:
#seting parameters that should be logged on MLFlow
#these parameters were used in feature engineering (inputing missing values)
#or parameters of the model (fit_intercept for Linear Regression model)
params = {
      "features drop": 'according to Capstone_features_Features.csv',
      "explanation": 'OLS unscaled',
      "csv used": 'Featureselection03.csv',
      "NaN handling": 'V.SLPOG.act.PRC and ME.SFCI.act.gPkWh filled with 0, rest dropped by row',
      'Shape' : df.shape,
      'Scaler' : ''
  }

In [ ]:
#logging params to mlflow
mlflow.log_params(params)
#setting tags
mlflow.set_tag("running_from_jupyter", "True")
#logging metrics
mlflow.log_metric("train-" + "RMSE", rmse_train)
mlflow.log_metric("test-" + "RMSE", rmse_test)
# logging the model to mlflow will not work without a AWS Connection setup.. too complex for now
# but possible if running mlflow locally
# mlflow.log_artifact("../models")
# mlflow.sklearn.log_model(reg, "model")
mlflow.end_run()

## Further Analysis

### What are the most important features?

In [ ]:
model.params